# RDKit C++ api basic examples with cling and xeus-cling kernel



- CERN's cling C++ interpreter https://github.com/root-project/cling
- QuantStack xeus-cling jupyter notebook kernel https://github.com/QuantStack/xeus-cling

## Add RDKit  to include and library paths

In [1]:
#pragma cling add_library_path("/opt/conda/lib")
#pragma cling add_include_path("/opt/conda/include/rdkit")

## Load methotrexate from SMILES and count its atoms


In [3]:
#include <GraphMol/GraphMol.h>
#include <GraphMol/SmilesParse/SmilesParse.h>

#pragma cling load("libRDKitSmilesParse.so")
#pragma cling load("libRDKitGraphMol.so")

std::string methotrexate_smi = "CN(Cc1cnc2nc(N)nc(N)c2n1)c1ccc(C(=O)N[C@@H](CCC(=O)O)C(=O)O)cc1";
std::unique_ptr<RDKit::ROMol> methotrexate(RDKit::SmilesToMol(methotrexate_smi));
std::cout << methotrexate->getNumAtoms() << " atoms" << std::endl;

33 atoms


# Calc some properties

In [4]:
#include <GraphMol/Descriptors/MolDescriptors.h>
#pragma cling load("libRDKitDescriptors.so")

double logp = RDKit::Descriptors::calcClogP(*methotrexate);
std::cout << "logP: " << logp << std::endl;

double e_mwt = RDKit::Descriptors::calcExactMW(*methotrexate);
std::cout << "molecular weight: " << e_mwt << std::endl;


logP: 0.2684
molecular weight: 454.171


# Calc inchi and inchi key

In [5]:
#include <GraphMol/inchi.h>

#pragma cling load("libRDKitInchi.so")
#pragma cling load("libRDKitRDInchiLib.so")

RDKit::ExtraInchiReturnValues tmp;
std::string inchi = RDKit::MolToInchi(*methotrexate, tmp);
std::cout << "InChI: " << inchi << std::endl;

std::string key = RDKit::InchiToInchiKey(inchi);
std::cout << "InChIKey: " << key << std::endl;

InChI: InChI=1S/C20H22N8O5/c1-28(9-11-8-23-17-15(24-11)16(21)26-20(22)27-17)12-4-2-10(3-5-12)18(31)25-13(19(32)33)6-7-14(29)30/h2-5,8,13H,6-7,9H2,1H3,(H,25,31)(H,29,30)(H,32,33)(H4,21,22,23,26,27)/t13-/m0/s1
InChIKey: FBOZXECLQNJBKD-ZDUSSCGKSA-N


## 2D coord generation and depiction

In [6]:
#include <GraphMol/MolDraw2D/MolDraw2DSVG.h>
#include <GraphMol/Depictor/RDDepictor.h>
#include <fstream>
#include <string>
#include "nlohmann/json.hpp"

#pragma cling load("libRDKitMolDraw2D.so")
#pragma cling load("libRDKitDepictor.so")

namespace ht
{
    struct html
    {   
        inline html(const std::string& content)
        {
            m_content = content;
        }
        std::string m_content;
    };

    nl::json mime_bundle_repr(const html& a)
    {
        auto bundle = nl::json::object();
        bundle["text/html"] = a.m_content;
        return bundle;
    }
}

In [7]:
// generate the 2D coordinates:
RDDepict::compute2DCoords(*methotrexate);

// // generate SVG as string
RDKit::MolDraw2DSVG drawer(400, 400);
drawer.drawMolecule(*methotrexate);
drawer.finishDrawing();
std::string svgs = drawer.getDrawingText();

// show the molecule
ht::html mol_svg(svgs);
mol_svg

<?xml version='1.0' encoding='iso-8859-1'?>
 
<!-- END OF HEADER -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<path class='atom-9' d='M 235.69 36.975
Q 235.956 36.2887, 236.591 35.9097
Q 237.227 35.5204, 238.108 35.5204
Q 239.204 35.5204, 239.818 36.1146
Q 240.433 36.7087, 240.433 37.7638
Q 240.433 38.8394, 239.634 39.8433
Q 238.845 40.8472, 237.206 42.0354
L 240.556 42.0354
L 240.556 42.8549
L 235.67 42.8549
L 235.67 42.1686
Q 237.022 41.2057, 237.821 40.4886
Q 238.63 39.7716, 239.019 39.1262
Q 239.409 38.4809, 239.409 37.815
Q 239.409 37.1184, 239.06 36.7292
Q 238.712 36.3399, 238.108 36.3399
Q 237.524 36.3399, 237.134 36.5755
Q 236.745 36.8111, 236.469 37.3336
L 235.69 36.975
' fill='#0000FF'/>
 
 
 
<path class='atom-12' d='M 235.69 171.389
Q 235.956 170.703, 236.591 170.324
Q 237.227 169.934, 238.108 169.934
Q 239.204 169.934, 239.818 170.528
Q 240.433 171.123, 240.433 172.178
Q 240.433 173.253, 239.634 174.257
Q 238.845 175.261, 237.206 176.449
L 240.556 176.449
L 240.556 177.269
L 235.67 177.269
L 235.67 176.583
Q 237.022 175.62, 237.821 174.903
Q 238.63 174.185, 239.019 173.54
Q 239.409 172.895, 239.409 172.229
Q 239.409 171.532, 239.06 171.143
Q 238.712 170.754, 238.108 170.754
Q 237.524 170.754, 237.134 170.989
Q 236.745 171.225, 236.469 171.747
L 235.69 171.389
' fill='#0000FF'/>
 
 
<path class='atom-20' d='M 182.764 199.915
Q 182.764 197.276, 184.068 195.802
Q 185.372 194.327, 187.808 194.327
Q 190.245 194.327, 191.549 195.802
Q 192.853 197.276, 192.853 199.915
Q 192.853 202.584, 191.533 204.105
Q 190.214 205.611, 187.808 205.611
Q 185.387 205.611, 184.068 204.105
Q 182.764 202.6, 182.764 199.915
M 187.808 204.369
Q 189.485 204.369, 190.385 203.252
Q 191.301 202.119, 191.301 199.915
Q 191.301 197.757, 190.385 196.671
Q 189.485 195.569, 187.808 195.569
Q 186.132 195.569, 185.216 196.655
Q 184.316 197.742, 184.316 199.915
Q 184.316 202.134, 185.216 203.252
Q 186.132 204.369, 187.808 204.369
' fill='#FF0000'/>
 
 
<path class='atom-26' d='M 85.7593 367.932
Q 85.7593 365.293, 87.063 363.819
Q 88.3668 362.345, 90.8035 362.345
Q 93.2403 362.345, 94.5441 363.819
Q 95.8478 365.293, 95.8478 367.932
Q 95.8478 370.602, 94.5285 372.123
Q 93.2093 373.628, 90.8035 373.628
Q 88.3823 373.628, 87.063 372.123
Q 85.7593 370.617, 85.7593 367.932
M 90.8035 372.386
Q 92.4798 372.386, 93.38 371.269
Q 94.2957 370.136, 94.2957 367.932
Q 94.2957 365.775, 93.38 364.688
Q 92.4798 363.586, 90.8035 363.586
Q 89.1273 363.586, 88.2116 364.673
Q 87.3114 365.759, 87.3114 367.932
Q 87.3114 370.151, 88.2116 371.269
Q 89.1273 372.386, 90.8035 372.386
' fill='#FF0000'/>
 
<path class='atom-27' d='M 27.5564 334.329
Q 27.5564 331.69, 28.8601 330.216
Q 30.1639 328.741, 32.6006 328.741
Q 35.0374 328.741, 36.3411 330.216
Q 37.6449 331.69, 37.6449 334.329
Q 37.6449 336.998, 36.3256 338.519
Q 35.0063 340.025, 32.6006 340.025
Q 30.1794 340.025, 28.8601 338.519
Q 27.5564 337.014, 27.5564 334.329
M 32.6006 338.783
Q 34.2769 338.783, 35.1771 337.666
Q 36.0928 336.532, 36.0928 334.329
Q 36.0928 332.171, 35.1771 331.085
Q 34.2769 329.983, 32.6006 329.983
Q 30.9244 329.983, 30.0087 331.069
Q 29.1084 332.156, 29.1084 334.329
Q 29.1084 336.548, 30.0087 337.666
Q 30.9244 338.783, 32.6006 338.783
' fill='#FF0000'/>
<path class='atom-29' d='M 143.962 199.915
Q 143.962 197.276, 145.266 195.802
Q 146.57 194.327, 149.006 194.327
Q 151.443 194.327, 152.747 195.802
Q 154.051 197.276, 154.051 199.915
Q 154.051 202.584, 152.731 204.105
Q 151.412 205.611, 149.006 205.611
Q 146.585 205.611, 145.266 204.105
Q 143.962 202.6, 143.962 199.915
M 149.006 204.369
Q 150.683 204.369, 151.583 203.252
Q 152.499 202.119, 152.499 199.915
Q 152.499 197.757, 151.583 196.671
Q 150.683 195.569, 149.006 195.569
Q 147.33 195.569, 146.415 196.655
Q 145.514 197.742, 145.514 199.915
Q 145.514 202.134, 146.415 203.252
Q 147.33 204.369, 149.006 204.369
' fill='#FF0000'/>
 
<path class='ato

## Morgan fingerprint and Tanimoto similarity

In [8]:
#include <GraphMol/Fingerprints/Fingerprints.h>
#include <GraphMol/Fingerprints/MorganFingerprints.h>
#include <DataStructs/ExplicitBitVect.h>
#include <DataStructs/BitOps.h>

#pragma cling load("libRDKitFingerprints.so")
#pragma cling load("libRDKitDataStructs.so")


std::string aspirin_smi = "CC(=O)Oc1ccccc1C(=O)O";
std::string paracetamol_smi = "CC(=O)Nc1ccc(O)cc1";

std::unique_ptr<RDKit::ROMol> aspirin(RDKit::SmilesToMol(aspirin_smi));
std::unique_ptr<RDKit::ROMol> paracetamol(RDKit::SmilesToMol(paracetamol_smi));

// unhashed fps
RDKit::SparseIntVect<boost::uint32_t> *fp_aspirin, *fp_paracetamol;

fp_aspirin = RDKit::MorganFingerprints::getFingerprint(*aspirin, 2);
fp_paracetamol = RDKit::MorganFingerprints::getFingerprint(*paracetamol, 2);

double tani = TanimotoSimilarity(*fp_aspirin, *fp_paracetamol);
std::cout << "Similarity between aspirin and paracetamol: " << tani << std::endl;


Similarity between aspirin and paracetamol: 0.27451


## Substructure search

In [9]:
#include <GraphMol/Substruct/SubstructUtils.h>
#pragma cling load("libRDKitSubstructMatch.so")

std::unique_ptr<RDKit::ROMol> mol_ss(RDKit::SmilesToMol("CC[C@H](F)Cl"));
std::unique_ptr<RDKit::ROMol> patt(RDKit::SmartsToMol("C[C@H](F)Cl"));

RDKit::MatchVectType res;

if( RDKit::SubstructMatch(*mol_ss , *patt , res) ) {
    std::cout << "SMARTS 1 match" << std::endl;
  } else {
    std::cout << "Not SMARTS 1 match" << std::endl;
}

res

SMARTS 1 match


{ {0 , 1}, {1 , 2}, {2 , 3}, {3 , 4} }

## Maximum Common Substructure

In [10]:
#include "GraphMol/FMCS/FMCS.h"
#pragma cling load("libRDKitFMCS.so")

std::vector<RDKit::ROMOL_SPTR> mols;

const std::string smi[] = {
    aspirin_smi,
    paracetamol_smi,
    methotrexate_smi
};

for (auto& i : smi) {
    mols.emplace_back(RDKit::SmilesToMol(i));
}

RDKit::MCSResult res = RDKit::findMCS(mols);
res.SmartsString

"[#6]1:[#6]:[#6]:[#6]:[#6]:[#6]:1"

# Murcko scaffold

In [11]:
#include "GraphMol/MolHash/MolHash.h"
#pragma cling load("libRDKitMolHash.so")

std::unique_ptr<RDKit::RWMol> murcko_mol(RDKit::SmilesToMol(methotrexate_smi));
auto scaffold = RDKit::MolHash::MolHash(murcko_mol.get(), RDKit::MolHash::HashFunction::MurckoScaffold);
std::cout << scaffold << std::endl;

std::unique_ptr<RDKit::RWMol> extended_murcko_mol(RDKit::SmilesToMol(methotrexate_smi));
auto extended_murcko = RDKit::MolHash::MolHash(extended_murcko_mol.get(), RDKit::MolHash::HashFunction::ExtendedMurcko);
std::cout << extended_murcko << std::endl;


c1ccc(NCc2cnc3ncncc3n2)cc1
*c1ccc(N(*)Cc2cnc3nc(*)nc(*)c3n2)cc1
